In [2]:
%env PYTORCH_ENABLE_MPS_FALLBACK=1

env: PYTORCH_ENABLE_MPS_FALLBACK=1


In [3]:
import os

# Set the environment variable to enable CPU fallback for MPS
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

# AlexNet - The paper which started the deep learning revolution!!!

AlexNet is a seminal paper in the field of deep learning, it was one of the first deep neural networks trained in the wild on a real
problem. ImageNet is the dataset this model trained on and the dataset is a classification task, with 1.2 million images and 1000 classes.

When AlexNet was released it changed the computer vision game, shifting focus from handcrafted specific methods to more general CV 
methods, i.e. neural networks. It was 10.3% ahead in top-1 against the next best competitor (top-1 means of the classes how many did it
get correct).

In this session I will provide you with an AlexNet cookbook to help you implement the model.

## Boiler plate code blocks

In [4]:
import torch
import torchvision

import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision.transforms as transforms

from torch.optim.lr_scheduler import ReduceLROnPlateau

In [5]:
if torch.backends.mps.is_available():
	device = torch.device("mps")
	x = torch.ones(1, device=device)
	print(x)
    
elif torch.backends.cuda.is_built():
	device = torch.device("cuda")
	x = torch.ones(1, device=device)
	print (x)
else:
	device = None
	print ("MPS device not found.")

tensor([1.], device='mps:0')


We will be using the ImageNette dataset (a subset of ImageNet as ImageNet is around 125gb so it's not feasible to run without a GPU and a lot of time)

To download ImageNette run these following commands:

mkdir Data && cd data 
wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz
tar -xvf imagenette2-160.tgz 


# The AlexNet CookBook
<h2><b> For today's session I will provide you with a list of ingredients, which will be all the layers you need </b></h2>

Beware some things are missing from this cookbook and you will need to utilise the paper to figure out the exact details.

AlexNet Implementation Ingredients list:

** n represents the number of total layers, so if n = 3 then there will be 3 of those layers **

- Convolutional layers:
	- conv1 - 11x11 @ 96 
	- conv2 - 5x5 @ 256
	- conv3 - 3x3 @ 384
	- conv4 - 3x3 @ 384
	- conv5 - 3x3 @ 256

- Fully connected layers:
	- fc1 - in_features = ?, out_features = 4096
	- fc2 - in_features = 4096, out_features = 4096
	- fc3 - in_features = 4096, out_features = 1000

- Non-Linear activation layers:
	- relu_n - ReLU layers follow every layer

- Norm layers: 
	- LRN_n - Follows specific layers, check the paper for more details or ask me :) - size=?, alpha=?, beta=?, k=?

- maxpool layers:
	- mp_n - Once again follows specific layers, check the paper for more details or ask me :)

And that is all you need to construct AlexNet!!!

In [6]:
class AlexNet(nn.Module):
	def __init__(self):
		super().__init__()
		
		self.conv1 = nn.Conv2d(
			kernel_size=(11, 11),
			in_channels=3,
			out_channels=96,
			stride=4,
		)
		self.relu1 = nn.ReLU()
		self.LRN1 = nn.LocalResponseNorm(
			size=5, alpha=10**(-4), beta=0.75, k=2
		)
		self.maxpool1 = nn.MaxPool2d(
			kernel_size=(3, 3),
			stride=(2, 2),
		)
		
		self.conv2 = nn.Conv2d(
			kernel_size=(5, 5),
			in_channels=96,
			out_channels=256,
			padding=2,
		)
		self.relu2 = nn.ReLU()
		self.LRN2 = nn.LocalResponseNorm(
			size=5, alpha=10**(-4), beta=0.75, k=2
		)
		self.maxpool2 = nn.MaxPool2d(
			kernel_size=(3, 3),
			stride=2,
		)
		
		self.conv3 = nn.Conv2d(
			kernel_size=(3, 3),
			in_channels=256,
			out_channels=384,
			padding=1,
		)
		self.relu3 = nn.ReLU()
		
		self.conv4 = nn.Conv2d(
			kernel_size=(3, 3),
			in_channels=384,
			out_channels=384,
			padding=1
		)
		self.relu4 = nn.ReLU()
		
		self.conv5 = nn.Conv2d(
			kernel_size=(3, 3),
			in_channels=384,
			out_channels=256,
			padding=1
		)
		self.relu5 = nn.ReLU()
		self.maxpool3 = nn.MaxPool2d(
			kernel_size=(3, 3),
			stride=2
		)
		
		self.fc1 = nn.Linear(9216, 4096)
		self.relu6 = nn.ReLU()
		self.dropout1 = nn.Dropout(p=0.5)
		
		self.fc2 = nn.Linear(4096, 4096)
		self.relu7 = nn.ReLU()
		self.dropout2 = nn.Dropout(p=0.5)
		
		self.fc3 = nn.Linear(4096, 1000)
		#self.fc3 = nn.Linear(4096, 10)  # If using CIFAR10
		# self.relu8 = nn.ReLU() is this supposed to be here?
		#self.softmax = nn.Softmax()
		
	def forward(self, x):
		# This block represents first convolution
		x = self.conv1(x)
		#print(f'Shape after conv1: {x.shape}')
		x = self.relu1(x)
		#print(f'Shape after ReLU1: {x.shape}')
		x = self.LRN1(x)
		#print(f'Shape after LRN1: {x.shape}')
		x = self.maxpool1(x)
		#print(f'Shape after MP1: {x.shape}\n')
		
		# Second convolution
		x = self.conv2(x)
		#print(f'Shape after conv2: {x.shape}')
		x = self.relu2(x)
		#print(f'Shape after ReLU2: {x.shape}')
		x = self.LRN2(x)
		#print(f'Shape after LRN2: {x.shape}')
		x = self.maxpool2(x)
		#print(f'Shape after MP2: {x.shape}\n')
		
		# Third convolution
		x = self.conv3(x)
		#print(f'Shape after conv3: {x.shape}')
		x = self.relu3(x)
		#print(f'Shape after ReLU3: {x.shape}\n')
		
		# Fourth convolution
		x = self.conv4(x)
		#print(f'Shape after conv4: {x.shape}')
		x = self.relu4(x)
		#print(f'Shape after relu4: {x.shape}\n')
		
		# Fifth convolution
		x = self.conv5(x)
		#print(f'Shape after conv5: {x.shape}')
		x = self.relu5(x)
		#print(f'Shape after relu5: {x.shape}')
		x = self.maxpool3(x)
		#print(f'Shape after MP3: {x.shape}\n')
		
		# Before passing to the fully connected layer we must flatten the tensor as nn.Linear expects a matrix (2d input)
		x = torch.flatten(x, start_dim=1)
		#print(x.shape)
		
		# First fully connected layer
		x = self.fc1(x)
		#print(f'Shape after fc1: {x.shape}')
		x = self.relu6(x)
		#print(f'Shape after relu6: {x.shape}')
		x = self.dropout1(x)
		#print(f'Shape after dropout1: {x.shape}\n')
	
		# Second fully connected layer
		x = self.fc2(x)
		#print(f'Shape after fc2: {x.shape}')
		x = self.relu7(x)
		#print(f'Shape after relu7: {x.shape}')
		x = self.dropout2(x)
		#print(f'Shape after dropout2: {x.shape}\n')
		
		# Final FC layer
		x = self.fc3(x)
		#print(f'Shape after fc3: {x.shape}')
		# x = self.relu8(x)
		#print(f'Shape after relu8: {x.shape}')
		#x = self.softmax(x)
		#print(f'Shape after softmax: {x.shape}\n')
		
		return x
	
model = AlexNet()

if device is not None:
	model.to(device)

In [7]:
# Define transformations
transform = transforms.Compose(
    [transforms.Resize((227, 227)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

TRAIN_DATA_DIR = './Data/imagenette2-160/train'
TEST_DATA_DIR = './Data/imagenette2-160/val'

# Load ImageNette dataset
trainset = torchvision.datasets.ImageFolder(
        TRAIN_DATA_DIR, transform=transform
    )
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)


testset= torchvision.datasets.ImageFolder(
        TEST_DATA_DIR, transform=transform
    )
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, weight_decay=0.0005, momentum=0.9)

# Define learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

for epoch in range(35):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}] training loss: {train_loss:.3f}')
    
    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data in test_loader:  # Assuming test_loader is used as a validation loader
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_loss = val_running_loss / len(test_loader)
    val_accuracy = 100 * val_correct / val_total
    print(f'Epoch [{epoch + 1}] validation loss: {val_loss:.3f}, accuracy: {val_accuracy:.2f}%')
    
    # Update the LR scheduler with validation loss
    scheduler.step(val_loss)
    print(f'LR: {scheduler.get_last_lr()}')

/opt/anaconda3/envs/Karpathy_NN_course/lib/python3.11/site-packages/torch/nn/functional.py:2592: UserWarning: The operator 'aten::avg_pool3d.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  div = avg_pool3d(div, (size, 1, 1), stride=1).squeeze(1)
